In [1]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.tools.retriever import create_retriever_tool
from langchain.prompts import SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate

with open('./openai_api_key.txt') as f:
    openai_api_key = f.read()

# Create Embedding LLM (OpenAI)

In [2]:
embedding_llm = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Load Databases

In [3]:
db = Chroma(persist_directory='./chroma', embedding_function=embedding_llm,
            collection_name="planetbucks")

# Creating RAG tools

In [4]:
retriever_tool = create_retriever_tool(
    db.as_retriever(search_type='mmr'),
    name = "planetbucks_search",
    description = """Search for information about PlanetBucks store, including general information, 
    coffee drink menus, specialty coffee beans menu, and bean fact sheet.""",
)

# Create Agent

In [5]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

In [6]:
system_message = """You are a helpful assistant working at a coffee shop.
You can use a given chat history and given tools to respond to a user.
Your character is a polite and friendly female.
You answer concisely, without introduction or appending.
If you do not know the answer, just say 'I do not know.'
When you calculate the price of an ordered item, you should think step-by-step.
"""

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=system_message)),
        MessagesPlaceholder(
            variable_name="chat_history", optional=True
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{input}"
        ),  # Where the human input will injected
        MessagesPlaceholder(
            variable_name="agent_scratchpad"
        ),  # Where the memory will be stored.
    ]
)

In [8]:
tools = [retriever_tool, ]

In [9]:
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_openai_functions_agent
from langchain.memory import ConversationBufferMemory

agent = create_openai_functions_agent(llm, tools, prompt,)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

# Usage

In [10]:
def wrapper_chat_history(chat_history, memory):
    chat_history = []
    for m in memory.chat_memory.messages:
        chat_history.append(m.content)
    return chat_history

In [11]:
def converse(message, chat_history):
    response = agent_executor.invoke({"input": message})
    chat_history = wrapper_chat_history(chat_history, memory)
    return response['output']

In [12]:
import gradio as gr

demo = gr.ChatInterface(fn=converse)

demo.launch(share=False)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...

Invoking: `planetbucks_search` with `{'query': 'store hours'}`




Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7


Shop Name: PlanetBucks
What we sell: coffee drinks and coffee bean

Operating time:
Monday to Friday 7 am - 8 pm
Saturday, Sunday, National Holiday 9 am - 4 pm

Address: ชั้น G ห้าง Emquartier สุขุมวิท กรุงเทพฯ
Branch 1: Icon Siam Mall, Third Floor, South Wing, Bangkok, Thailand
Branch 2: 103th Floor, Empire State Building, NYC

Payment: cash, credit card, square, bitcoin

﻿Table of Drink Menu,,
Coffee Drink,Small Size Price ($),Large Size Price ($)
Espresso,2.00,3.00
Americano,2.50,3.50
Cappuccino,3.00,4.50
Latte,3.50,5.00
Mocha,4.00,5.50
,,
Table of Add-on items or change,,
Ice,0.50,
Oat milk,1.00,
Soy milk,0.75,

Geisha Coffee Beans - Hacienda La Esmeralda

Origin: Boquete, Panama, from Hacienda La Esmeralda, renowned for producing award-winning Geisha coffee.
Altitude: Grown at elevations of 1,500 to 2,000 meters, ideal for developing the bean's distinctive flavors.
Taste Profile: A harmonious blend of jasmine and bergamot aromas, complemented by flavors of honey, peaches, and wild

Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7


...

﻿Table of Drink Menu,,
Coffee Drink,Small Size Price ($),Large Size Price ($)
Espresso,2.00,3.00
Americano,2.50,3.50
Cappuccino,3.00,4.50
Latte,3.50,5.00
Mocha,4.00,5.50
,,
Table of Add-on items or change,,
Ice,0.50,
Oat milk,1.00,
Soy milk,0.75,

Blue Mountain Coffee Beans - The Jewel of Jamaica

Origin: Cultivated in the revered Blue Mountains of Jamaica, from estates that meet the highest standards of quality.
Altitude: Thrives at altitudes of 910 to 1,700 meters, where the climate is perfect for developing its signature flavors.
Taste Profile: A rare blend of sweet herbs and nuts, complemented by a chocolate undertone. It boasts a mild acidity, a smooth body, and a clean, sweet aftertaste.
Processing: Wet-processed to enhance its smooth and clean profile.
Roast Level: Medium roast, optimizing its mild and unique flavors.
Brewing Recommendations: Ideal for drip coffee makers and French press, where its subtle complexities can be savored.
Packaging: 16 oz (454g) vacuum-sealed bags fo

Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7


﻿Table of Drink Menu,,
Coffee Drink,Small Size Price ($),Large Size Price ($)
Espresso,2.00,3.00
Americano,2.50,3.50
Cappuccino,3.00,4.50
Latte,3.50,5.00
Mocha,4.00,5.50
,,
Table of Add-on items or change,,
Ice,0.50,
Oat milk,1.00,
Soy milk,0.75,

Blue Mountain Coffee Beans - The Jewel of Jamaica

Origin: Cultivated in the revered Blue Mountains of Jamaica, from estates that meet the highest standards of quality.
Altitude: Thrives at altitudes of 910 to 1,700 meters, where the climate is perfect for developing its signature flavors.
Taste Profile: A rare blend of sweet herbs and nuts, complemented by a chocolate undertone. It boasts a mild acidity, a smooth body, and a clean, sweet aftertaste.
Processing: Wet-processed to enhance its smooth and clean profile.
Roast Level: Medium roast, optimizing its mild and unique flavors.
Brewing Recommendations: Ideal for drip coffee makers and French press, where its subtle complexities can be savored.
Packaging: 16 oz (454g) vacuum-sealed bags fo

Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7


Blue Mountain Coffee Beans - The Jewel of Jamaica

Origin: Cultivated in the revered Blue Mountains of Jamaica, from estates that meet the highest standards of quality.
Altitude: Thrives at altitudes of 910 to 1,700 meters, where the climate is perfect for developing its signature flavors.
Taste Profile: A rare blend of sweet herbs and nuts, complemented by a chocolate undertone. It boasts a mild acidity, a smooth body, and a clean, sweet aftertaste.
Processing: Wet-processed to enhance its smooth and clean profile.
Roast Level: Medium roast, optimizing its mild and unique flavors.
Brewing Recommendations: Ideal for drip coffee makers and French press, where its subtle complexities can be savored.
Packaging: 16 oz (454g) vacuum-sealed bags for maximum freshness.
Price: $40 per bag, a testament to its unparalleled quality and heritage.
Certifications: Officially certified as 100% Blue Mountain Coffee, ensuring authenticity and excellence.
Awards: Renowned worldwide for its exceptional q